In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot as plt
import numpy as np
import torch
import pandas as pd
import random
import matplotlib.pyplot as plt
import time
from sklearn import metrics

In [ ]:
# GPU or CPU
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')
print(device)

cpu


### Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CAI 2024_Flexible Building Space Usage Opt/Dataset/dataset_merged_r1+r2.csv')

# occupancy
data_occ_presence = data['occupant_presence [binary]'].to_numpy()
data_occ_count = data['occupant_count [number]'].to_numpy()

# energy use
data_energy_1 = data['ceiling_fan_energy [kWh]'].to_numpy()
data_energy_2 = data['lighting_energy [kWh]'].to_numpy()
data_energy_3 = data['plug_load_energy [kWh]'].to_numpy()
data_energy_4 = data['chilled_water_energy [kWh]'].to_numpy()
data_energy_5 = data['fcu_fan_energy [kWh]'].to_numpy()

# indoor data
data_t = data['air_temperature [Celsius]'].to_numpy()
data_h = data['indoor_relative_humidity [%]'].to_numpy()

# outdoor
data_outdoor_t = data['dry_bulb_temp [Celsius]'].to_numpy()
data_outdoor_h = data['outdoor_relative_humidity [%]'].to_numpy()

# time
data_month = data['Month'].to_numpy()
data_time_1 = data['Week'].to_numpy()
data_time_2 = data['Hour'].to_numpy()

In [ ]:
# indoor: x1 - x3
x1 = np.reshape(data_occ_count,(-1,1))
x2 = np.reshape(data_t,(-1,1))
x3 = np.reshape(data_h,(-1,1))

# outdoor: x11 - x12
x11 = np.reshape(data_outdoor_t,(-1,1))
x12 = np.reshape(data_outdoor_h,(-1,1))


# **********************
X = np.hstack((x1,x2,x3,x11,x12))
print('X_shape\n',X.shape)
# **********************

# y1 - y3
y1 = np.reshape(data_energy_1,(-1,1))
y2 = np.reshape(data_energy_2,(-1,1))
y3 = np.reshape(data_energy_3,(-1,1))
y4 = np.reshape(data_energy_4,(-1,1))
y5 = np.reshape(data_energy_5,(-1,1))

Y = np.hstack((y1,y2,y3,y4,y5))
#print('Y_shape\n',Y.shape)

Y_sum = y1 + y2 + y3 + y4 + y5
#print('Y_sum_shape\n',Y_sum.shape)

X_shape
 (16650, 5)


In [ ]:
# Training set: 0 - 60%
train_size = int(len(X) * 0.6)
train_range = train_size

# Validation set: 60% - 80%
valid_size = int(len(X) * 0.2)
valid_range = train_size + valid_size

# Testing set: 80% - 100%
test_size = int(len(X) * 0.2)
test_range = train_size + valid_size + test_size

#print(train_range)
#print(valid_range)
#print(test_range)

X_training, Y_training, Y_sum_training = X[:train_range ,:], Y[:train_range ,:], Y_sum[:train_range ,:]
print('X_training\n', X_training.shape, ' | ', '0 ', train_range)
#print('Y_training\n', Y_training.shape)
#print('Y_sum_training\n', Y_sum_training.shape)

X_valid, Y_valid, Y_sum_valid = X[train_range:valid_range ,:], Y[train_range:valid_range ,:], Y_sum[train_range:valid_range ,:]
print('X_valid\n', X_valid.shape, ' | ', train_range, valid_range)
#print('Y_valid\n', Y_valid.shape)
#print('Y_sum_valid\n', Y_sum_valid.shape)

X_test, Y_test, Y_sum_test = X[valid_range: ,:], Y[valid_range: ,:], Y_sum[valid_range: ,:]
print('X_test\n', X_test.shape,  ' | ', valid_range, test_range)
#print('Y_test\n', Y_test.shape)
#print('Y_sum_test\n', Y_sum_test.shape)


X_training
 (9990, 5)  |  0  9990
X_valid
 (3330, 5)  |  9990 13320
X_test
 (3330, 5)  |  13320 16650


### XGBRegressor model

In [ ]:
model=xgb.XGBRegressor(max_depth=8,n_estimators=1396,
                       learning_rate=0.115,early_stopping_rounds=5)
model.fit(X_training,Y_sum_training,
          eval_set=[(X_valid, Y_sum_valid)])


[0]	validation_0-rmse:0.60026
[1]	validation_0-rmse:0.57968
[2]	validation_0-rmse:0.56407
[3]	validation_0-rmse:0.55223
[4]	validation_0-rmse:0.54331
[5]	validation_0-rmse:0.53679
[6]	validation_0-rmse:0.53227
[7]	validation_0-rmse:0.53031
[8]	validation_0-rmse:0.52792
[9]	validation_0-rmse:0.52761
[10]	validation_0-rmse:0.52630
[11]	validation_0-rmse:0.52587
[12]	validation_0-rmse:0.52654
[13]	validation_0-rmse:0.52621
[14]	validation_0-rmse:0.52671
[15]	validation_0-rmse:0.52788


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.115, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1396, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### Main program

In [ ]:
score = model.score(X_training,Y_sum_training)
print("Training score: ", score)

prection_test = model.predict(X_test)
# MSE Loss
mse_loss = metrics.mean_squared_error(prection_test,Y_sum_test)
print('Test-loss:','%.3f'%mse_loss)

Training score:  0.48952268369601326
Test-loss: 0.264


In [ ]:
#model.save_model('/content/drive/MyDrive/CAI 2024_Flexible Building Space Usage Opt/ML model/xgboost_office.model')